In [39]:
import os
os.environ['PATH'] += ':/opt/local/bin'
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [66]:
torch.manual_seed(42)
ins = torch.randint(low=0, high=10, size=(100,))
outs = torch.zeros(len(nums))
for i, j in enumerate(nums[:-1]):
    outs[i] = nums[i].item() + nums[i+1].item()


In [41]:
x = nums[:10]
y = outs[:10]
for i in range(len(x) - 1):
    print(i)
    context1 = x[i]
    context2 = x[i+1]
    answer = y[i]
    print(f"when context is {context1} + {context2} it equals {answer}")

0
when context is 2 + 7 it equals 9.0
1
when context is 7 + 6 it equals 13.0
2
when context is 6 + 4 it equals 10.0
3
when context is 4 + 6 it equals 10.0
4
when context is 6 + 5 it equals 11.0
5
when context is 5 + 0 it equals 5.0
6
when context is 0 + 4 it equals 4.0
7
when context is 4 + 0 it equals 4.0
8
when context is 0 + 3 it equals 3.0


In [42]:
block_size = 2
batch_size = 4


# FIRST ITERATION, NOT THE RIGHT APPROACH, I FIGURED I COULD MAKE TWO TENSORS, ONE WITH THE NUMBERS AND THE OTHER
# JUST OFFSET BY 1
# def get_batch():
#     torch.manual_seed(42)
#     ix = torch.randint(len(nums) - block_size, (batch_size,),)
#     Xb = nums[ix]
#     Yb = outs[ix]
# #     x = torch.stack([nums[i:i+block_size] for i in ix])
# #     y = torch.stack([nums[i+1:i+block_size+1] for i in ix])
    
#     print(Xb)
#     print(Yb)
#     return Xb, Yb
    

# get_batch()

In [67]:

def get_batch(nums):
    X, Y = [], []
    for i in range(len(nums) - 2):
        context = []
        context.append(nums[i])
        context.append(nums[i+1])
        X.append(context)
        Y.append(outs[i].long())



    X = torch.tensor(X).long()
    Y = torch.tensor(Y).long()
    return X, Y


n1 = int(.9*(len(ins)))
training = ins[:n1]
test = ins[n1:]


Xtr, Ytr = get_batch(training)
Xtest, Ytest = get_batch(test)
print(Xtr[0], Ytr[0])

tensor([2, 7]) tensor(9)


In [68]:
import torch
import torch.nn as nn
import torch.optim as optim


# Define your neural network model
class Calc(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Calc, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.fc(x)

# Prepare your data 
Xtr, Ytr = get_batch(training)

# Define your model
model = Calc(input_dim=2, output_dim=1)

# Define a loss function
criterion = nn.MSELoss()

# Choose an optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    predictions = model(Xtr)
    
    # Compute the loss
    loss = criterion(predictions, Ytr)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float